In [1]:
import sys
sys.path.append("../src")

In [2]:
from model import UnetModel
from skimage.io import imshow
%matplotlib inline

Using TensorFlow backend.


In [3]:
pretrained = True
if True:
    losses = []
    for size in [(1024, 1024), (128, 128), (256, 256)]:
        for batch_size in [1, 2]:
            for fold_num in range(1, 2):
                batches =  16 * 100 // batch_size
                model = UnetModel(f"unet__{size}_{batch_size}_{fold_num}_augm_", input_size=size, patience=10)
                if pretrained:
                    model.unet.load_weights(f"./unet__{size}_{batch_size}_{fold_num}_augm.best.weights.h5py")
                print(f"Size = {size}, batch size = {batch_size}")
                if True:
                    learn_rates = [0.00001, 0.000001]
                    epochs_list = [50, 50]
                else:
                    learn_rates = [0.0001, 0.00001, 0.000001]
                    epochs_list = [50, 50, 50]
                pretrained = False
                for lr, epochs in zip(learn_rates, epochs_list):
                    train_loss, val_loss = model.fit_resized(lr, epochs, fold_num, batch_size=batch_size, batches=batches)
                losses.append((train_loss, val_loss))

../src/unet.py:64: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("in...)`
  model = Model(input=inputs, outputs=conv12)


Size = (1024, 1024), batch size = 1


../src/model.py:53: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=1600, validation_data=<generator..., validation_steps=800, callbacks=[<callback..., epochs=50)`
  callbacks=self.callbacks)


Epoch 1/50
1600/1600 [==============================] - 1744s - loss: -0.9884 - dice_coef: 0.9935 - val_loss: -0.9691 - val_dice_coef: 0.9853
Epoch 2/50
  15/1600 [..............................] - ETA: 1518s - loss: -0.9882 - dice_coef: 0.9937

KeyboardInterrupt: 

In [ ]:
img_size = (1024, 1024)
model = UnetModel("prediction", input_size=img_size)
model.unet.load_weights(f"./unet__{img_size}_1_1_augm.best.weights.h5py")
from generator import train_generator_resized, valid_generator_resized
batch = next(valid_generator_resized(n_fold=1, size=img_size, batch_size=1))

In [ ]:
imshow(model.unet.predict(batch[0])[0].squeeze() * 255)

In [ ]:
imshow(batch[1][0].squeeze().astype('uint8'))

In [ ]:
from predictor import Predictor
predictor = Predictor(model.unet)
threshold = predictor.find_threshold(img_size, 1)
predictor.create_resized_submission(img_size, threshold=threshold)

In [ ]:
import numpy as np

In [ ]:
for i in np.arange(0.40, 1.00, 0.05):
    print(i)